# Environment setup

In [ ]:
import pandas as pd
import pprint
import os
from langchain_community.document_loaders import DataFrameLoader

In [ ]:
import os
import google.generativeai as genai
from langchain_google_genai import ChatGoogleGenerativeAI

genai.configure(api_key=os.getenv('GOOGLE_API_KEY'))

# Filter generated testset

In [ ]:
import pandas as pd
from datasets import Dataset

# Load the dataset
eval_dataset = Dataset.load_from_disk("eval_dataset")

# Convert to DataFrame
eval_df_syntethic = eval_dataset.to_pandas()
eval_df_syntethic = eval_df_syntethic[["question", "answer", "source_doc", "context", "chunk_num"]]
print(len(eval_df_syntethic))
display(eval_df_syntethic.head())

In [ ]:
import pandas as pd
from datasets import Dataset

# Load the dataset
eval_dataset_adjacent_chunks = Dataset.load_from_disk("eval_dataset_adjacent_chunks")

# Convert to DataFrame
eval_df_adjacent_chunks_syntethic = eval_dataset_adjacent_chunks.to_pandas()
eval_df_adjacent_chunks_syntethic = eval_df_adjacent_chunks_syntethic[["question", "answer", "source_doc", "context", "chunk_num"]]
print(len(eval_df_adjacent_chunks_syntethic))
display(eval_df_adjacent_chunks_syntethic.head())

In [ ]:
import pandas as pd
from datasets import Dataset

# Load the dataset
eval_dataset_random_chunks = Dataset.load_from_disk("eval_dataset_random_chunks")

# Convert to DataFrame
eval_df_random_chunks_syntethic = eval_dataset_random_chunks.to_pandas()
eval_df_random_chunks_syntethic = eval_df_random_chunks_syntethic[["question", "answer", "source_doc", "context", "chunk_num"]]
print(len(eval_df_random_chunks_syntethic))
display(eval_df_random_chunks_syntethic.head())

In [ ]:
# REMOVE ROWS WITH NAN CHUNK NUM 
# Remove rows where 'column_name' contains NaN values
eval_df_syntethic = eval_df_syntethic[eval_df_syntethic['chunk_num'].notna()]

# Optionally, reset the index of the new dataframe (to avoid gaps in index after removal)
eval_df_syntethic = eval_df_syntethic.reset_index(drop=True)

# Assuming df is your dataframe
eval_df_syntethic['chunk_num'] = eval_df_syntethic['chunk_num'].apply(lambda x: [int(x)])

# Print the resulting dataframe
print(len(eval_df_syntethic))
display(eval_df_syntethic)

In [ ]:
# Concatenate the DataFrames
df = pd.concat([
    eval_df_random_chunks_syntethic, 
    eval_df_syntethic, 
    eval_df_adjacent_chunks_syntethic
], ignore_index=True)

display(df)

# Big LLM - Gemini 1.5 pro latest to filter the good QA pairs

In [ ]:
from langchain.schema import HumanMessage, SystemMessage
import pandas as pd
from datasets import Dataset
from langchain_google_genai import ChatGoogleGenerativeAI
from tqdm import tqdm 

# Initialize the Gemini model
model_gemini = ChatGoogleGenerativeAI(
    model="gemini-1.5-pro-latest",
    temperature=0
)

tqdm.pandas()

def evaluate_pair(row):
    # Costruisci un input strutturato per il modello
    messages = [
    SystemMessage(content="Sei un critico che valuta coppie di domande e risposte per una FAQ di un software gestionale. \
        Le coppie di domande e risposte devono soddisfare i seguenti criteri per essere considerate utili per valutare un chatbot destinato al supporto clienti di un software gestionale: \
        1. **Rilevanza**: Devono affrontare temi rilevanti per gli utenti di un software gestionale. \
        2. **Logicità e utilità**: Devono essere logiche e utili per fornire informazioni chiare e pratiche agli utenti. \
        Valuta la seguente coppia e decidi se è utile per testare un chatbot per il supporto clienti. Fornisci un feedback strutturato e dettagliato seguendo il formato specificato."),
    HumanMessage(content=f"**Domanda:** {row['question']}\n**Risposta:** {row['answer']}\n\nValuta questa coppia di domanda-risposta e fornisci un feedback nel seguente formato:\n\
        [Sì/No] \
                \
        Spiegazione delle risposta:\
        - **Motivazione del perchè la coppia domanda-risposta è considerata utile o meno** \
        - **Motivazione sulla rilevanza**: [Breve spiegazione, se rilevante o non rilevante] \
        - **Motivazione sulla logicità**: [Breve spiegazione, se logica e utile o meno] \founda
        Indica anche eventuali miglioramenti necessari nella domanda o nella risposta.")
    ]

    # Invia l'input strutturato al modello
    response = model_gemini(messages)
    return response.content[:8]  # Estrai l'output del modello

# Apply the evaluation function to each row
df["feedback"] = df.progress_apply(evaluate_pair, axis=1)
display(df)

In [ ]:
# Save the results to a CSV file
df = pd.read_csv("filtered_testset_withGemini.csv")
df

In [ ]:
# Filter rows where 'Sì' is present in the 'feedback' column
filtered_testset = df[df['feedback'].str.contains('Sì', na=False)]

# Reset the index if desired
filtered_testset = filtered_testset.reset_index(drop=True)
filtered_testset = filtered_testset.drop(columns="feedback")
filtered_testset = filtered_testset.drop(columns="source_doc")

# Display the filtered dataframe
display(filtered_testset)

In [ ]:
import random
extract = random.randint(0, len(filtered_testset))
pprint.pprint(filtered_testset['question'][extract])
pprint.pprint(filtered_testset['answer'][extract])

# Import also semantic testset

In [ ]:
import pandas as pd
from datasets import Dataset

# Load the dataset
eval_semantic_dataset = Dataset.load_from_disk("eval_semantic_dataset_random_chunks")

# Convert to DataFrame
eval_semantic_df_syntethic = eval_semantic_dataset.to_pandas()
eval_semantic_df_syntethic = eval_semantic_df_syntethic[["question", "answer", "context", "chunk_num"]]
print(len(eval_semantic_df_syntethic))
display(eval_semantic_df_syntethic.head())

# Ensemble of smaller LLMs, small models to filter good QA pairs

In [ ]:
!huggingface-cli login

In [ ]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer1 = AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-3B")
model1 = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-3.2-3B")

tokenizer2 = AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-3B-Instruct")
model2 = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-3.2-3B-Instruct")

tokenizer3 = AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-1B")
model3 = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-3.2-1B")

tokenizer4 = AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-1B-Instruct")
model4 = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-3.2-1B-Instruct")

# Mixed evaluation set to reduce the bias in choosing the chunking

In [ ]:
# Evaluation testset created with chunks from recursive splitting technique
df_recursive_splitting =  pd.read_csv('filtered_matching_questions.csv')
df_recursive_splitting = df_recursive_splitting.drop(columns = "source_doc")
df_recursive_splitting

In [ ]:
import random
extract = random.randint(0, len(df_recursive_splitting))
pprint.pprint(df_recursive_splitting['question'][extract])
pprint.pprint(df_recursive_splitting['answer'][extract])
print("\n\n-------------------------------------------------------------------------------\nContext:")
pprint.pprint(df_recursive_splitting['context'][extract])

In [ ]:
import pandas as pd
from datasets import Dataset

# Load the dataset
eval_semantic_dataset = Dataset.load_from_disk("eval_semantic_dataset_random_chunks")

# Convert to DataFrame
eval_semantic_df_syntethic = eval_semantic_dataset.to_pandas()
eval_semantic_df_syntethic = eval_semantic_df_syntethic[["question", "answer", "context", "chunk_num"]]
print(len(eval_semantic_df_syntethic))
display(eval_semantic_df_syntethic)

In [ ]:
import random
extract = random.randint(0, len(eval_semantic_df_syntethic))
pprint.pprint(eval_semantic_df_syntethic['question'][extract])
pprint.pprint(eval_semantic_df_syntethic['answer'][extract])
print("\n\n-------------------------------------------------------------------------------\nContext:")
pprint.pprint(eval_semantic_df_syntethic['context'][extract])

In [ ]:
# Concatenate the DataFrames
df_semantic = pd.concat([
    eval_semantic_df_syntethic
], ignore_index=True)

display(df_semantic)

In [ ]:
# Filter the semantic testset using a big LLM as Gemini-1.5
from langchain.schema import HumanMessage, SystemMessage
import pandas as pd
from datasets import Dataset
from langchain_google_genai import ChatGoogleGenerativeAI
from tqdm import tqdm 

# Initialize the Gemini model
model_gemini = ChatGoogleGenerativeAI(
    model="gemini-1.5-pro-latest",
    temperature=0
)

tqdm.pandas()

def evaluate_pair(row):
    # Costruisci un input strutturato per il modello
    messages = [
        SystemMessage(content="Sei un critico che valuta coppie di domande e risposte per una FAQ di un software gestionale. \
        Le coppie di domande e risposte devono soddisfare i seguenti criteri per essere considerate utili per valutare un chatbot destinato al supporto clienti di un software gestionale: \
        1. **Rilevanza**: Devono affrontare temi rilevanti per gli utenti di un software gestionale. \
        2. **Logicità e utilità**: Devono essere logiche e utili per fornire informazioni chiare e pratiche agli utenti. \
        Valuta la seguente coppia e decidi se è utile per testare un chatbot per il supporto clienti. Fornisci un feedback strutturato e dettagliato seguendo il formato specificato."),
        HumanMessage(content=(
            f"**Domanda:** {row['question']}\n**Risposta:** {row['answer']}\n\n"
            "Valuta questa coppia di domanda-risposta e fornisci un feedback nel seguente formato:\n"
            "[Sì/No]\n"
            "Spiegazione delle risposta:\n"
            "- **Motivazione del perchè la coppia domanda-risposta è considerata utile o meno**\n"
            "- **Motivazione sulla rilevanza**: [Breve spiegazione, se rilevante o non rilevante]\n"
            "- **Motivazione sulla logicità**: [Breve spiegazione, se logica e utile o meno]\n"
            "Indica anche eventuali miglioramenti necessari nella domanda o nella risposta."
        ))
    ]

    # Invia l'input strutturato al modello
    response = model_gemini(messages)
    return response.content[:8]  # Estrai l'output del modello


# Apply the evaluation function to each row
df_semantic["feedback"] = df_semantic.progress_apply(evaluate_pair, axis=1)
display(df_semantic)

In [ ]:
df_semantic.to_csv("filtered_semantic_testset_withGemini", index=False)

In [ ]:
# Filter rows where 'Sì' is present in the 'feedback' column
filtered_semantic_testset = df_semantic[df_semantic['feedback'].str.contains('Sì', na=False)]

# Reset the index if desired
filtered_semantic_testset = filtered_semantic_testset.reset_index(drop=True)
filtered_semantic_testset = filtered_semantic_testset.drop(columns="feedback")

# Display the filtered dataframe
display(filtered_semantic_testset)

In [ ]:
import random
extract = random.randint(0, len(filtered_semantic_testset))
pprint.pprint(filtered_semantic_testset['question'][extract])
pprint.pprint(filtered_semantic_testset['answer'][extract])
print("\n\n-------------------------------------------------------------------------------\nContext:")
pprint.pprint(filtered_semantic_testset['context'][extract])

In [ ]:
# Random data extraction for final mix evaluation set
# Assume df is your DataFrame
random_50_rows_recursive_splitting = df_recursive_splitting.sample(n=50, random_state=42, ignore_index=True)

# Display the extracted rows
display(random_50_rows_recursive_splitting)

In [ ]:
# Random data extraction for final mix evaluation set
# Assume df is your DataFrame
random_50_rows_semantic_splitting = filtered_semantic_testset.sample(n=50, random_state=42, ignore_index=True)

# Display the extracted rows
display(random_50_rows_semantic_splitting)

In [ ]:
# Concatenate the DataFrames
final_df = pd.concat([
    random_50_rows_recursive_splitting, 
    random_50_rows_semantic_splitting
], ignore_index=True)

display(final_df)

In [ ]:
final_df.to_csv("final_mixed_recursive_semantic_evaluation_set.csv", index=False)